# attr examples

    '''sh
    pip install attrs
    '''
`@attrs` — это аннотация, которая сообщает, что этот класс будет объявлен с помощью библиотеки `attrs`. Затем каждый атрибут нужно определить как `attrib()`, потому что нет необходимости использовать метод `__init__()`. После этого мы можем просто создать экземпляр класса точно так же, как если бы существовал метод `__init__`.

[arrts examles](https://github.com/python-attrs/attrs/blob/master/docs/examples.rst)

In [37]:
# ! pip install unidecode


In [55]:
from attr import attrs, attrib, asdict, astuple, fields, validators
from numbers import Number

from unidecode import unidecode


In [56]:
STUDENTS_ID_LEN = 6
SEX_VARIANTS = ['man', 'woman']
MIN_AGE = 10
MAX_AGE = 64

In [105]:
@attrs
class Person(object):
    name: str = attrib()
    age: int = attrib(validator=validators.instance_of(int))
    sex: str = attrib(default='man')
    skills: list = attrib(factory=list)
    projects: list = attrib(factory=list)
#     projects = attrib(init=False)
   
    
    @age.validator
    def check_age(self, attribute, value):
        if  MIN_AGE > value > MAX_AGE:
            raise ValueError(f'wrong age {value}')
    
    @sex.validator
    def check_sex(self, attribute, value):
        if value not in SEX_VARIANTS:
            raise ValueError(f'wrong sex {value}')
    
    
            
    def __str__(self):
        result = f'{self.name} / {self.sex} {self.age}y.o. '
        if self.skills:
            result +=  ', '.join(self.skills)
        if self.projects:
            result += ', '.join(self.projects)
        return result

In [136]:
# @attrs(frozen=True)
@attrs()
class User(object):
    user_id: str = attrib(default='a')
    password: str = attrib(repr=False, default='')
    # password = attrib(repr=lambda value: '***')
    
    def get_user_id(self):
        return self.user_id

In [137]:
@attrs
class Student(Person, User):
    student_id: str = attrib(default='000000')
    group_id: str = attrib(default='a0')
    course: int = attrib()
    
    @course.default
    def _course(self):
        result = self.age - 17
        if result <= 5:
            return result
        return 1
            
    
    @student_id.validator
    def check_student_id(self, attribute, value):
        if len(str(value)) != STUDENTS_ID_LEN:
            raise ValueError(f'student_id must be 6 characters! got {len(str(value))}')
        if not value.isdigit():
            raise ValueError('student_id must be numbers')
    
    def get_student_id(self):
        return self.student_id
    
    def __str__(self):
        return super().__str__() + f'/ id: {self.student_id} / group: {self.group_id} / uid: {self.user_id}'


In [138]:
susy = Person(name="Susy", age=18, sex="woman")

print(susy)

Susy / woman 18y.o. 


In [139]:
sus = Person(name="Sus", age=16)

In [140]:
sus.skills =  ['python', 'django']
print(sus)

Sus / man 16y.o. python, django


In [141]:
student = Student(name='Cyr', age=37, sex='man', user_id='cyr', student_id='123456', group_id='b12')
        

In [142]:
print(asdict(student))
print(astuple(student))
student #password not in repr

{'name': 'Cyr', 'age': 37, 'sex': 'man', 'skills': [], 'projects': [], 'user_id': 'cyr', 'password': '', 'student_id': '123456', 'group_id': 'b12', 'course': 1}
('Cyr', 37, 'man', [], [], 'cyr', '', '123456', 'b12', 1)


Student(name='Cyr', age=37, sex='man', skills=[], projects=[], user_id='cyr', student_id='123456', group_id='b12', course=1)

In [143]:
print(student)

Cyr / man 37y.o. / id: 123456 / group: b12 / uid: cyr


In [144]:
student_two = Student(**asdict(susy), user_id='susy', student_id='123496', group_id='b12')

In [145]:
print(student_two)

Susy / woman 18y.o. / id: 123496 / group: b12 / uid: susy


In [146]:
for a in fields(Student):
    print(a.name, a.type)

name <class 'str'>
age <class 'int'>
sex <class 'str'>
skills <class 'list'>
projects <class 'list'>
user_id <class 'str'>
password <class 'str'>
student_id <class 'str'>
group_id <class 'str'>
course <class 'int'>
